# Introduction to Quantum Computing Machine Learnin for Finance

###### Imports

In [ ]:
!pip install pennylane
!pip install matplotlib
!pip install jax==0.4.23 jaxlib==0.4.23
!pip install qiskit
!pip install qiskit_ibm_runtime
!pip install numpy
!pip install IPython
!pip install qiskit_machine_learning
!pip install seaborn
!pip install pandas
!pip install scikit-learn
!pip install pennylane 
!pip install qiskit_ibm_runtime
!pip install jax==0.4.23 jaxlib==0.4.23 --force-reinstall
!pip install pennylane jax==0.4.23 jaxlib==0.4.23
!pip install "numpy<2.0"

## What is a Qubit?

A Qubit is a Quantum Bit, which can hold the superposition of both 0 and 1. Below is an example of when a Qubit is set to absolute values. Qubits are visualised using Bloch Spheres as shown below.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
 
qc = QuantumCircuit(2)
qc.h(0)
qc.x(1)
 
state = Statevector(qc)
plot_bloch_multivector(state)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# ---- STEP 0: Load data ----
FTSE = pd.read_csv('Combined_FTSE_100_Data.csv')
BOND = pd.read_csv('Combined_UK_10-Year_Bond_Yield_Data.csv')

# ---- STEP 1: Parse dates ----
FTSE['Date'] = pd.to_datetime(FTSE['Date'])
BOND['Date'] = pd.to_datetime(BOND['Date'])

# ---- STEP 2: Set 'Date' as index for direct daily join ----
FTSE = FTSE.set_index('Date')
BOND = BOND.set_index('Date')

# ---- STEP 3: Clean column names if needed ----
# Check columns to adapt this line based on your file structure
print(BOND.columns)

# Example adjustment:
# BOND['BOND_Value'] = BOND['End month level of yield for 10-year nominal zero coupon [a] IUMMNZC']
# If your column name is different, replace accordingly:

BOND['BOND_Value'] = BOND.iloc[:, 0]  # assumes first column is yield value
BOND = BOND[['BOND_Value']]  # keep only cleaned yield column

# ---- STEP 4: Join datasets on daily dates ----
joined = FTSE.join(BOND, how='inner')  # 'inner' ensures only overlapping dates

# ---- STEP 5: Prepare Close values and drop unused columns ----
# Remove commas and convert to float
joined['Price'] = joined['Price'].str.replace(',', '', regex=False).astype(float)

# Now assign 'Close'
joined['Close'] = joined['Price']

# Drop unused columns
joined = joined.drop(['Open', 'High', 'Low', 'Price'], axis='columns', errors='ignore')


# ---- STEP 6: Feature engineering ----
joined['BondChange'] = joined['BOND_Value'].diff().fillna(0)
joined['FTSE_MA'] = joined['Close'].rolling(window=3).mean().bfill()

cutoff_date = joined.index.max() - pd.DateOffset(years=5)
joined_5y = joined.loc[joined.index >= cutoff_date]


# ---- STEP 7: Shuffle and split for baseline classical model (1D input) ----
data = shuffle(joined_5y.copy(), random_state=4)

X = data[['BOND_Value']].to_numpy()
Y = data['Close'].to_numpy()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

# ---- STEP 8: Subset for quantum kernel or 2D experiments ----
sdata = joined.query("BOND_Value >= 3.95").copy()
sdata = shuffle(sdata, random_state=4)

Xs = sdata[['BOND_Value', 'BondChange']].to_numpy()
Ys = sdata['Close'].to_numpy()

# ---- STEP 9: Scale features to [0, π] for quantum encoding ----
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.5, random_state=0)

scaler = MinMaxScaler(feature_range=(0, np.pi))
Xs_scaled = scaler.fit_transform(Xs)
Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs_scaled, Ys, test_size=0.5, random_state=0)




## Classical SVR

The Classical Models are Support Vector Machines, using Radial Basis Function (Squared Exponential), Linear and Polynomial kernels

### Full Dataset

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

C = 10000
epsilon = 0.9
lw = 2

svr_rbf = SVR(kernel="rbf", C=C, gamma="scale", epsilon=epsilon)
svr_lin = SVR(kernel="linear", C=C, gamma="scale", epsilon=epsilon)
svr_poly = SVR(kernel="poly", C=C, gamma="scale", epsilon=epsilon)

SVR_models = [svr_rbf, svr_lin, svr_poly]
kernel_label = ["RBF", "Linear", "Poly"]
model_color = ["m", "c", "g"]

# Plotting
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6), sharey=True)

for ix, svr in enumerate(SVR_models):
    preds = svr.fit(X_train, Y_train).predict(X_test)
    r2 = r2_score(Y_test, preds)

    axes[ix].scatter(X_test, preds, color=model_color[ix], lw=lw, label=f"{kernel_label[ix]} model\nR² = {r2:.2f}")
    axes[ix].scatter(X_train, Y_train, facecolor="none", edgecolor='m', s=50, label="Train")
    axes[ix].scatter(X_test, Y_test, facecolor="none", edgecolor="g", s=50, label="True")

    axes[ix].legend(loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=1, fancybox=True, shadow=True)
    axes[ix].set_title(f"{kernel_label[ix]} Kernel")

# Global labels
fig.text(0.5, 0.04, "UK Treasury Bond Yields IUMLPNY", ha="center")
fig.text(0.06, 0.5, "FTSE 100 Close values", va="center", rotation="vertical")
fig.suptitle("Support Vector Regression (1D, BOND_Value only, C=10000)", fontsize=16)
plt.tight_layout()
plt.show()

# Save predictions
rbf_pred = svr_rbf.predict(X_test)
lin_pred = svr_lin.predict(X_test)
poly_pred = svr_poly.predict(X_test)


### Subset Dataset

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.metrics import r2_score

# Classical SVR models for full and subset
C = 10000
epsilon = 0.9
kernel_label = ["RBF", "Linear", "Poly"]
model_color = ["m", "c", "g"]
lw = 2

# Define classical models
models = [SVR(kernel="rbf", C=C, epsilon=epsilon),
          SVR(kernel="linear", C=C, epsilon=epsilon),
          SVR(kernel="poly", C=C, epsilon=epsilon)]

# Fit and predict for both datasets
full_preds, full_r2 = [], []
subset_preds, subset_r2 = [], []

for model in models:
    # Full dataset
    model.fit(X_train, Y_train)
    full_pred = model.predict(X_test)
    full_preds.append(full_pred)
    full_r2.append(r2_score(Y_test, full_pred))

    # Subset dataset
    model.fit(Xs_train, Ys_train)
    subset_pred = model.predict(Xs_test)
    subset_preds.append(subset_pred)
    subset_r2.append(r2_score(Ys_test, subset_pred))

# Plotting classical SVRs (2x3)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10), sharey=True)

for i in range(3):
    # Full dataset (top row)
    axes[0, i].scatter(X_test, full_preds[i], color=model_color[i], lw=lw, label=f"Predicted\nR² = {full_r2[i]:.2f}")
    axes[0, i].scatter(X_test, Y_test, facecolor="none", edgecolor="g", s=40, label="True")
    axes[0, i].set_title(f"{kernel_label[i]} - Full Dataset")
    axes[0, i].legend()

    # Subset dataset (bottom row)
    axes[1, i].scatter(Xs_test[:, 0], subset_preds[i], color=model_color[i], lw=lw, label=f"Predicted\nR² = {subset_r2[i]:.2f}")
    axes[1, i].scatter(Xs_test[:, 0], Ys_test, facecolor="none", edgecolor="g", s=40, label="True")
    axes[1, i].set_title(f"{kernel_label[i]} - Subset")
    axes[1, i].legend()

axes[0, 0].set_ylabel("FTSE Close (Full)")
axes[1, 0].set_ylabel("FTSE Close (Subset)")
fig.text(0.5, 0.04, "UK Treasury Bond Yields (scaled)", ha="center")
fig.suptitle("Classical SVR Comparison: Full Dataset vs Subset", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()





## Quantum Support Vector Regression (QSVR)
We utilise a quantum circuit to create a quantum kernel

### Quantum Circuit

In [ ]:
import pennylane as qml
dev = qml.device("default.qubit", wires=2)

def feature_map(x):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.RX(x[0] + 0.3, wires=0)
    qml.RY(x[1] + 0.7, wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RZ(x[0] * x[1], wires=1)
    qml.CNOT(wires=[0, 1])

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.expval(qml.PauliZ(0))

def quantum_kernel(X1, X2):
    X1 = np.array(X1, dtype=float)
    X2 = np.array(X2, dtype=float)
    return np.array([[kernel_circuit(x1, x2) for x2 in X2] for x1 in X1])

from pennylane import draw

# Example inputs
x1 = [0.5, 1.2]
x2 = [0.1, 0.9]

# Draw the circuit
drawer = qml.draw(kernel_circuit)
print(drawer(x1, x2))



### QSVR Subset

In [ ]:
Xs_train_poly = np.hstack([Xs_train, Xs_train**2])
Xs_test_poly = np.hstack([Xs_test, Xs_test**2])

Xs_train_poly = Xs_train_poly.astype(float)
Xs_test_poly = Xs_test_poly.astype(float)

K_train = quantum_kernel(Xs_train_poly, Xs_train_poly)
K_test = quantum_kernel(Xs_test_poly, Xs_train_poly)

svr = SVR(kernel='precomputed', C=10000, epsilon=0.1)
svr.fit(K_train, Ys_train)

sqsvr_pred = svr.predict(K_test)
r2 = r2_score(Ys_test, sqsvr_pred)
print("R² score (QSVR with polynomial features):", r2)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(Ys_test, label='True', marker='o')
plt.plot(sqsvr_pred, label='QSVR Prediction', marker='x')
plt.title("Quantum SVR Prediction vs True Close Prices Subset")
plt.xlabel("Sample")
plt.ylabel("FTSE Close")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

r2_qsvr_subset = r2_score(Ys_test, sqsvr_pred)

plt.figure(figsize=(10, 6))
plt.scatter(Xs_test_poly[:, 0], sqsvr_pred, color='orange', label="QSVR Prediction", marker='x', s=60)
plt.scatter(Xs_test_poly[:, 0], Ys_test, facecolor="none", edgecolor="blue", label="True Values", s=60)

plt.title(f"Quantum SVR Prediction (Polynomial Features) Subset\nR² = {r2_qsvr_subset:.2f}", fontsize=14)
plt.xlabel("UK Treasury Bond Yields (scaled, poly)", fontsize=12)
plt.ylabel("FTSE 100 Close values", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### QSVR Full Dataset

In [74]:
X_train_poly = np.hstack([X_train, X_train**2])
X_test_poly = np.hstack([X_test, X_test**2])

K_train_full = quantum_kernel(X_train_poly, X_train_poly)
svr_q_full = SVR(kernel='precomputed', C=10000, epsilon=0.1).fit(K_train_full, Y_train)

K_test_full = quantum_kernel(X_test_poly, X_train_poly)
sqsvr_pred_full = svr_q_full.predict(K_test_full)
r2_qsvr_full = r2_score(Y_test, sqsvr_pred_full)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(Y_test, label='True', marker='o')
plt.plot(sqsvr_pred_full, label='QSVR Prediction', marker='x')
plt.title("Quantum SVR Prediction vs True Close Prices Full Dataset")
plt.xlabel("Sample")
plt.ylabel("FTSE Close")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
r2_qsvr_full = r2_score(Y_test, sqsvr_pred_full)

plt.figure(figsize=(10, 6))
plt.scatter(X_test_poly[:, 0], sqsvr_pred_full, color='orange', label="QSVR Prediction", marker='x', s=60)
plt.scatter(X_test_poly[:, 0], Y_test, facecolor="none", edgecolor="blue", label="True Values", s=60)

plt.title(f"Quantum SVR Prediction (Polynomial Features) Full\nR² = {r2_qsvr_full:.2f}", fontsize=14)
plt.xlabel("UK Treasury Bond Yields (scaled, poly)", fontsize=12)
plt.ylabel("FTSE 100 Close values", fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


###### imports

In [ ]:
!pip install tensorflow

## Variational Quantum Regression

In [ ]:
import pennylane as qml
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import logging

# Suppress TensorFlow warnings
logging.getLogger('tensorflow').setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')

# Step 1: Feature engineering and scaling
poly = PolynomialFeatures(2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

pca = PCA(n_components=min(5, X_train_poly.shape[1]))
X_train_pca = pca.fit_transform(X_train_poly)
X_test_pca = pca.transform(X_test_poly)

x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train_pca)
X_test_scaled = x_scaler.transform(X_test_pca)

# Step 2: Target normalization
y_scaler = StandardScaler()
Y_train_scaled = y_scaler.fit_transform(Y_train.reshape(-1, 1)).flatten()
Y_test_scaled = y_scaler.transform(Y_test.reshape(-1, 1)).flatten()

# Step 3: Quantum model definition
n_qubits = X_train_scaled.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="tf", diff_method="backprop")
def circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(0))]  # Now returns shape (1,)


# Wrap into a KerasLayer
weight_shapes = {"weights": (3, n_qubits, 3)}
qlayer = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=1, dtype=tf.float32)

# Step 4: Define model using Functional API
inputs = tf.keras.Input(shape=(n_qubits,), dtype=tf.float32)
x = qlayer(inputs)
x = tf.keras.layers.Dense(64, activation='swish', kernel_regularizer=l2(0.01))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01))(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Step 5: Compile the model
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.0005,
    decay_steps=100
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber())
early_stop = EarlyStopping(monitor="loss", patience=10, restore_best_weights=True)

# Step 6: Train the model
X_input = tf.convert_to_tensor(X_train_scaled.astype(np.float32).reshape(-1, n_qubits))
Y_input = tf.convert_to_tensor(Y_train_scaled.astype(np.float32).reshape(-1, 1))
history = model.fit(X_input, Y_input, epochs=150, batch_size=8, verbose=1, callbacks=[early_stop])

# Step 7: Plot training loss
plt.plot(history.history['loss'])
plt.title("Enhanced VQR Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Huber Loss")
plt.grid(True)
plt.show()

# Step 8: Evaluate the model
X_test_input = tf.convert_to_tensor(X_test_scaled.astype(np.float32).reshape(-1, n_qubits))
Y_pred_scaled = model.predict(X_test_input).flatten()
Y_pred = y_scaler.inverse_transform(Y_pred_scaled.reshape(-1, 1)).flatten()
Y_test_orig = y_scaler.inverse_transform(Y_test_scaled.reshape(-1, 1)).flatten()

r2 = r2_score(Y_test_orig, Y_pred)
print("✅ Final Enhanced VQR R² score (original scale):", r2)

# Step 9: Visual comparison
plt.figure(figsize=(6, 4))
plt.bar(["Classical SVR Best", "QSVR", "Final VQR"], [0.81, 0.05, r2], color=["green", "blue", "purple"])
plt.axhline(0, color='gray', linestyle='--')
plt.ylabel("R² Score")
plt.title("📊 Model Comparison: Classical vs Quantum")
plt.ylim(-0.1, 1.0)
plt.grid(True, axis='y')
plt.show()


### VQR Subset

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

Xs_train_subset = pnp.array(Xs_train)
Xs_test_subset = pnp.array(Xs_test)
Ys_train_subset = pnp.array(Ys_train)
Ys_test_subset = pnp.array(Ys_test)

x_scaler_subset = MinMaxScaler(feature_range=(0, pnp.pi))
Xs_train_scaled_subset = x_scaler_subset.fit_transform(Xs_train_subset)
Xs_test_scaled_subset = x_scaler_subset.transform(Xs_test_subset)

y_scaler_subset = MinMaxScaler()
Ys_train_scaled_subset = y_scaler_subset.fit_transform(Ys_train_subset.reshape(-1, 1)).flatten()
Ys_test_scaled_subset = y_scaler_subset.transform(Ys_test_subset.reshape(-1, 1)).flatten()

n_qubits = Xs_train.shape[1]
layers = 3
dev = qml.device("default.qubit", wires=n_qubits)

def variational_circuit(x, weights):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    for l in range(layers):
        for i in range(n_qubits):
            qml.RY(weights[l, i], wires=i)
            qml.RZ(weights[l, i], wires=i)
        if l % 2 == 0:
            qml.CNOT(wires=[0, 1])
        else:
            qml.CNOT(wires=[1, 0])

@qml.qnode(dev)
def circuit(x, weights):
    variational_circuit(x, weights)
    return qml.expval(qml.PauliZ(0))

def predict(X, weights):
    return pnp.array([circuit(x, weights) for x in X])

def loss(weights, X, Y):
    preds = predict(X, weights)
    return pnp.mean((preds - Y) ** 2)

pnp.random.seed(42)
weights_subset = pnp.array(pnp.random.uniform(0, pnp.pi, size=(layers, n_qubits)), requires_grad=True)
opt = qml.AdamOptimizer(stepsize=0.1)
epochs = 300

for i in range(epochs):
    weights_subset = opt.step(lambda w: loss(w, Xs_train_scaled_subset, Ys_train_scaled_subset), weights_subset)
    if i % 20 == 0:
        l = loss(weights_subset, Xs_train_scaled_subset, Ys_train_scaled_subset)
        print(f"Epoch {i}: Loss = {l:.4f}")

Ys_pred_scaled_subset = predict(Xs_test_scaled_subset, weights_subset)
Ys_pred_subset_vqr = y_scaler_subset.inverse_transform(pnp.array(Ys_pred_scaled_subset).reshape(-1, 1)).flatten()
r2_vqr_subset = r2_score(Ys_test_subset, Ys_pred_subset_vqr)

plt.figure(figsize=(8, 5))
plt.plot(Ys_test_subset, label='True', marker='o')
plt.plot(Ys_pred_subset_vqr, label='VQR Prediction', marker='x', color='orange')
plt.title("PennyLane VQR Prediction vs True Close Prices (Subset)")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE Close")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 5))
plt.scatter(Xs_test_subset[:, 0], Ys_pred_subset_vqr, color='orange', marker='x', label=f"VQR Subset\nR² = {r2_vqr_subset:.2f}")
plt.scatter(Xs_test_subset[:, 0], Ys_test_subset, facecolor="none", edgecolor="g", s=50, label="True")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE 100 Close values")
plt.title("PennyLane VQR (Subset Dataset)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### VQR Full Dataset

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

X_train_full = pnp.array(X_train)
X_test_full = pnp.array(X_test)
Y_train_full = pnp.array(Y_train)
Y_test_full = pnp.array(Y_test)

x_scaler_full = MinMaxScaler(feature_range=(0, pnp.pi))
X_train_scaled_full = x_scaler_full.fit_transform(X_train_full)
X_test_scaled_full = x_scaler_full.transform(X_test_full)

y_scaler_full = MinMaxScaler()
Y_train_scaled_full = y_scaler_full.fit_transform(Y_train_full.reshape(-1, 1)).flatten()
Y_test_scaled_full = y_scaler_full.transform(Y_test_full.reshape(-1, 1)).flatten()

n_qubits = X_train_full.shape[1]
layers = 3
dev = qml.device("default.qubit", wires=n_qubits)

def variational_circuit(x, weights):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)
    for l in range(layers):
        for i in range(n_qubits):
            qml.RY(weights[l, i], wires=i)
            qml.RZ(weights[l, i], wires=i)
        if n_qubits > 1:
            if l % 2 == 0:
                qml.CNOT(wires=[0, 1])
            else:
                qml.CNOT(wires=[1, 0])

@qml.qnode(dev)
def circuit(x, weights):
    variational_circuit(x, weights)
    return qml.expval(qml.PauliZ(0))

def predict(X, weights):
    return pnp.array([circuit(x, weights) for x in X])

def loss(weights, X, Y):
    preds = predict(X, weights)
    return pnp.mean((preds - Y) ** 2)

pnp.random.seed(42)
weights_full = pnp.array(pnp.random.uniform(0, pnp.pi, size=(layers, n_qubits)), requires_grad=True)
opt = qml.AdamOptimizer(stepsize=0.1)
epochs = 300

for i in range(epochs):
    weights_full = opt.step(lambda w: loss(w, X_train_scaled_full, Y_train_scaled_full), weights_full)
    if i % 20 == 0:
        l = loss(weights_full, X_train_scaled_full, Y_train_scaled_full)
        print(f"Epoch {i}: Loss = {l:.4f}")

Y_pred_scaled_full = predict(X_test_scaled_full, weights_full)
Y_pred_full_vqr = y_scaler_full.inverse_transform(pnp.array(Y_pred_scaled_full).reshape(-1, 1)).flatten()
r2_vqr_full = r2_score(Y_test_full, Y_pred_full_vqr)

plt.figure(figsize=(8, 5))
plt.plot(Y_test_full, label='True', marker='o')
plt.plot(Y_pred_full_vqr, label='VQR Prediction', marker='x', color='orange')
plt.title("PennyLane VQR Prediction vs True Close Prices (Full Dataset)")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE Close")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 5))
plt.scatter(X_test_full[:, 0], Y_pred_full_vqr, color='orange', marker='x', label=f"VQR Full\nR² = {r2_vqr_full:.2f}")
plt.scatter(X_test_full[:, 0], Y_test_full, facecolor="none", edgecolor="g", s=50, label="True")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE 100 Close values")
plt.title("PennyLane VQR (Full Dataset)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



# Final Plot

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(30, 10))
gs = GridSpec(2, 5, figure=fig)  # 5 columns: 3 classical + 1 QSVR + 1 VQR

qsvr_color = 'purple'
vqr_color = 'darkorange'

for i in range(3):
    ax_top = fig.add_subplot(gs[0, i])
    ax_top.scatter(X_test, full_preds[i], color=model_color[i], lw=2, label="Predicted")
    ax_top.scatter(X_test, Y_test, facecolor="none", edgecolor="g", s=40, label="True")
    ax_top.set_title(f"{kernel_label[i]} - Full")
    ax_top.legend(loc="upper left", fontsize=9)
    ax_top.set_ylabel("FTSE Close (£) ")
    ax_top.set_xlabel("UK Treasury Bond Yields (scaled) (%)")

    ax_bot = fig.add_subplot(gs[1, i])
    ax_bot.scatter(Xs_test[:, 0], subset_preds[i], color=model_color[i], lw=2, label="Predicted")
    ax_bot.scatter(Xs_test[:, 0], Ys_test, facecolor="none", edgecolor="g", s=40, label="True")
    ax_bot.set_title(f"{kernel_label[i]} - Subset")
    ax_bot.legend(loc="upper left", fontsize=9)
    ax_bot.set_ylabel("FTSE Close (£)")
    ax_bot.set_xlabel("UK Treasury Bond Yields (scaled) (%)")

ax_qsvr_full = fig.add_subplot(gs[0, 3])
ax_qsvr_full.scatter(X_test_poly[:, 0], sqsvr_pred_full, color=qsvr_color, lw=2, label="QSVR")
ax_qsvr_full.scatter(X_test_poly[:, 0], Y_test, facecolor="none", edgecolor="g", s=40)
ax_qsvr_full.set_title("QSVR - Full")
ax_qsvr_full.legend(loc="upper left", fontsize=9)
ax_qsvr_full.set_ylabel("FTSE Close (£)")
ax_qsvr_full.set_xlabel("UK Treasury Bond Yields (scaled) (%)")

ax_qsvr_sub = fig.add_subplot(gs[1, 3])
ax_qsvr_sub.scatter(Xs_test_poly[:, 0], sqsvr_pred, color=qsvr_color, lw=2, label="QSVR")
ax_qsvr_sub.scatter(Xs_test_poly[:, 0], Ys_test, facecolor="none", edgecolor="g", s=40)
ax_qsvr_sub.set_title("QSVR - Subset")
ax_qsvr_sub.legend(loc="upper left", fontsize=9)
ax_qsvr_sub.set_ylabel("FTSE Close (£)")
ax_qsvr_sub.set_xlabel("UK Treasury Bond Yields (scaled) (%)")

ax_vqr_full = fig.add_subplot(gs[0, 4])
ax_vqr_full.scatter(X_test[:, 0], Y_pred_full_vqr, color=vqr_color, marker='x', label="VQR")
ax_vqr_full.scatter(X_test[:, 0], Y_test, facecolor="none", edgecolor="g", s=40)
ax_vqr_full.set_title("VQR - Full")
ax_vqr_full.legend(loc="upper left", fontsize=9)
ax_vqr_full.set_ylabel("FTSE Close (£)")
ax_vqr_full.set_xlabel("UK Treasury Bond Yields (scaled) (%)")

ax_vqr_sub = fig.add_subplot(gs[1, 4])
ax_vqr_sub.scatter(Xs_test[:, 0], Ys_pred_subset_vqr, color=vqr_color, marker='x', label="VQR")
ax_vqr_sub.scatter(Xs_test[:, 0], Ys_test, facecolor="none", edgecolor="g", s=40)
ax_vqr_sub.set_title("VQR - Subset")
ax_vqr_sub.legend(loc="upper left", fontsize=9)
ax_vqr_sub.set_ylabel("FTSE Close (£)")
ax_vqr_sub.set_xlabel("UK Treasury Bond Yields (scaled) (%)")

fig.suptitle("Classical SVRs vs Quantum SVR vs VQR: Full & Subset Comparison", fontsize=18, y=1.03)
plt.tight_layout(rect=[0.03, 0.05, 1, 0.95])
plt.show()


# Evaluations

## Subset

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

model_names = ["RBF - Full", "Linear - Full", "Poly - Full", "QSVR - Full", "VQR - Full",
               "RBF - Subset", "Linear - Subset", "Poly - Subset", "QSVR - Subset", "VQR - Subset"]

predictions = [
    full_preds[0], full_preds[1], full_preds[2], sqsvr_pred_full, Y_pred_full_vqr,
    subset_preds[0], subset_preds[1], subset_preds[2], sqsvr_pred, Ys_pred_subset_vqr
]

true_values = [
    Y_test, Y_test, Y_test, Y_test, Y_test,
    Ys_test, Ys_test, Ys_test, Ys_test, Ys_test
]

mse_list = []
mae_list = []
r2_list = []

for pred, true in zip(predictions, true_values):
    mse_list.append(mean_squared_error(true, pred))
    mae_list.append(mean_absolute_error(true, pred))
    r2_list.append(r2_score(true, pred))

results_df = pd.DataFrame({
    "Model": model_names,
    "MSE": mse_list,
    "MAE": mae_list,
    "R²": r2_list
})

results_df = results_df.sort_values(by="R²", ascending=False).reset_index(drop=True)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="R²", y="Model")
plt.title("Model R² Comparison")
plt.show()
plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="MSE", y="Model")
plt.title("Model R² Comparison")
plt.show()
plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="MAE", y="Model")
plt.title("Model R² Comparison")
plt.show()



In [ ]:
from scipy.stats import ttest_rel
import pandas as pd

model_preds = {
    "SVR (RBF)": full_preds[0],
    "SVR (Linear)": full_preds[1],
    "SVR (Poly)": full_preds[2],
    "Quantum SVR": sqsvr_pred_full,
    "VQR": Y_pred_full_vqr,
}

true_y = Y_test

model_names = list(model_preds.keys())
p_values = []

for i in range(len(model_names)):
    for j in range(i+1, len(model_names)):
        name1, name2 = model_names[i], model_names[j]
        preds1, preds2 = model_preds[name1], model_preds[name2]
        
        
        t_stat, p_val = ttest_rel(preds1 - true_y, preds2 - true_y)
        
        p_values.append({
            "Model 1": name1,
            "Model 2": name2,
            "p-value": p_val
        })

ttest_results = pd.DataFrame(p_values)

ttest_results = ttest_results.sort_values(by="p-value").reset_index(drop=True)

fig, ax = plt.subplots(figsize=(10, 3)) 
ax.axis("off")
ax.table(cellText=ttest_results.values,
         colLabels=ttest_results.columns,
         loc="center",
         cellLoc="center",  
         colColours=["lightblue"] * len(ttest_results.columns))  
plt.title("p-value < 0.05 means statisitcally significant difference")
plt.show()

## Full Dataset

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Setup model labels
model_names = ["RBF - Full", "Linear - Full", "Poly - Full", "QSVR - Full", "VQR - Full",
               "RBF - Subset", "Linear - Subset", "Poly - Subset", "QSVR - Subset", "VQR - Subset"]

# Collect all predictions and ground truths (make sure your predictions are available)
predictions = [
    full_preds[0], full_preds[1], full_preds[2], sqsvr_pred_full, Y_pred_full_vqr,
    subset_preds[0], subset_preds[1], subset_preds[2], sqsvr_pred, Ys_pred_subset_vqr
]

true_values = [
    Y_test, Y_test, Y_test, Y_test, Y_test,
    Ys_test, Ys_test, Ys_test, Ys_test, Ys_test
]

# Initialize empty lists to store metrics
mse_list = []
mae_list = []
r2_list = []

# Compute metrics
for pred, true in zip(predictions, true_values):
    mse_list.append(mean_squared_error(true, pred))
    mae_list.append(mean_absolute_error(true, pred))
    r2_list.append(r2_score(true, pred))

# Create DataFrame
results_df = pd.DataFrame({
    "Model": model_names,
    "MSE": mse_list,
    "MAE": mae_list,
    "R²": r2_list
})

# Sort by R² descending
results_df = results_df.sort_values(by="R²", ascending=False).reset_index(drop=True)

display(results_df)

# Plot the results for R², MSE, and MAE
plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="R²", y="Model")
plt.title("Model R² Comparison")
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="MSE", y="Model")
plt.title("Model MSE Comparison")
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x="MAE", y="Model")
plt.title("Model MAE Comparison")
plt.show()


# Conclusion

## Final Conclusions

This experiment compared classical and quantum-enhanced regression techniques on financial time series data. The results are summarized below:

- **Classical SVR models** (RBF, Linear, Polynomial) provided strong baselines for predicting FTSE 100 index values based on UK bond yields.
- **Quantum SVR (QSVR)** using a custom quantum kernel achieved comparable R² scores, especially on structured subsets of the data, suggesting potential for **quantum-enhanced feature extraction**.
- **Variational Quantum Regressor (VQR)** demonstrated the feasibility of hybrid quantum-classical learning. While performance did not yet surpass classical SVRs, the architecture lays the groundwork for trainable quantum pipelines.

 **Key Insight:** Quantum methods are still maturing, but they show promise in capturing nonlinear dependencies when paired with careful feature engineering and domain-specific tuning.

 **Next Steps:**
- Scale experiments to larger datasets using simulators or real quantum backends.
- Incorporate noise-aware quantum layers.
- Explore multi-output regression or classification tasks with quantum-enhanced models.



In [ ]:
# ===============================
# 📦 IMPORTS
import os
import numpy as np
import matplotlib.pyplot as plt

# Ensure figures directory exists
os.makedirs("figures", exist_ok=True)

# ===============================
# 📊 CLASSICAL SVR PLOTS (FULL + SUBSET) – using saved predictions

kernel_label = ["RBF", "Linear", "Poly"]
# Updated model colors: RBF=magenta, Linear=cyan, Poly=purple
model_color = ["m", "c", "purple"]

# Plot FULL dataset SVR predictions
for i, pred in enumerate(full_preds):
    r2 = r2_score(Y_test, pred)
    plt.figure(figsize=(6,4))
    plt.scatter(X_test[:,0], Y_test, facecolor="none", edgecolor="g", s=50, label="True")
    plt.scatter(X_test[:,0], pred, alpha=0.6, color=model_color[i], label=f"Predicted\nR²={r2:.2f}")
    plt.xlabel("UK Treasury Bond Yields (scaled)")
    plt.ylabel("FTSE Close")
    plt.title(f"SVR ({kernel_label[i]} kernel, Full)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"figures/svr_{kernel_label[i].lower()}_full.png", dpi=300)
    plt.close()

# Plot SUBSET dataset SVR predictions
for i, pred in enumerate(subset_preds):
    r2 = r2_score(Ys_test, pred)
    plt.figure(figsize=(6,4))
    plt.scatter(Xs_test[:,0], Ys_test, facecolor="none", edgecolor="g", s=50, label="True")
    plt.scatter(Xs_test[:,0], pred, alpha=0.6, color=model_color[i], label=f"Predicted\nR²={r2:.2f}")
    plt.xlabel("UK Treasury Bond Yields (scaled)")
    plt.ylabel("FTSE Close")
    plt.title(f"SVR ({kernel_label[i]} kernel, Subset)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"figures/svr_{kernel_label[i].lower()}_subset.png", dpi=300)
    plt.close()

print("✅ Classical SVR plots saved with updated Poly SVR colour (purple).")


# ===============================
# 🌐 QSVR plots (FULL + SUBSET) using precomputed predictions

# FULL QSVR plot
plt.figure(figsize=(6,4))
plt.scatter(X_test[:,0], Y_test, facecolor="none", edgecolor="g", s=50, label="True")
plt.scatter(X_test[:,0], sqsvr_pred_full, alpha=0.6, color="brown", label=f"Predicted\nR²={r2_score(Y_test, sqsvr_pred_full):.2f}")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE Close")
plt.title("QSVR (Full Dataset)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("figures/qsvr_full.png", dpi=300)
plt.close()

# SUBSET QSVR plot
plt.figure(figsize=(6,4))
plt.scatter(Xs_test[:,0], Ys_test, facecolor="none", edgecolor="g", s=50, label="True")
plt.scatter(Xs_test[:,0], sqsvr_pred, alpha=0.6, color="brown", label=f"Predicted\nR²={r2_score(Ys_test, sqsvr_pred):.2f}")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE Close")
plt.title("QSVR (Subset Dataset)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("figures/qsvr_subset.png", dpi=300)
plt.close()

print("✅ QSVR plots saved.")

# ===============================
# 🔮 VQR plots (FULL + SUBSET) using precomputed predictions

# FULL VQR plot
plt.figure(figsize=(6,4))
plt.scatter(X_test[:,0], Y_test, facecolor="none", edgecolor="g", s=50, label="True")
plt.scatter(X_test[:,0], Y_pred_full_vqr, alpha=0.6, color="orange", label=f"Predicted\nR²={r2_score(Y_test, Y_pred_full_vqr):.2f}")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE Close")
plt.title("VQR (Full Dataset)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("figures/vqr_full.png", dpi=300)
plt.close()

# SUBSET VQR plot
plt.figure(figsize=(6,4))
plt.scatter(Xs_test[:,0], Ys_test, facecolor="none", edgecolor="g", s=50, label="True")
plt.scatter(Xs_test[:,0], Ys_pred_subset_vqr, alpha=0.6, color="orange", label=f"Predicted\nR²={r2_score(Ys_test, Ys_pred_subset_vqr):.2f}")
plt.xlabel("UK Treasury Bond Yields (scaled)")
plt.ylabel("FTSE Close")
plt.title("VQR (Subset Dataset)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("figures/vqr_subset.png", dpi=300)
plt.close()

print("✅ VQR plots saved.")

# ===============================
print("🎉 ALL MODEL PLOTS GENERATED AND SAVED TO figures/ DIRECTORY.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml

# Load your existing training data (ensure X_train_poly is defined as in your QSVR pipeline)
# Example (adapt to your variables if different):
# X_train_poly = np.hstack([X_train, X_train**2])

# Define quantum kernel circuit as implemented
dev = qml.device("default.qubit", wires=2)

def feature_map(x):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.RX(x[0] + 0.3, wires=0)
    qml.RY(x[1] + 0.7, wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RZ(x[0] * x[1], wires=1)
    qml.CNOT(wires=[0, 1])

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.expval(qml.PauliZ(0))

def quantum_kernel(X1, X2):
    X1 = np.array(X1, dtype=float)
    X2 = np.array(X2, dtype=float)
    return np.array([[kernel_circuit(x1, x2) for x2 in X2] for x1 in X1])

# Calculate kernel matrix
print("Computing quantum kernel matrix...")
K_train = quantum_kernel(X_train_poly, X_train_poly)

# Plot heatmap
plt.figure(figsize=(8,6))
plt.imshow(K_train, cmap='viridis')
plt.title("QSVR Quantum Kernel Matrix (Training Data)")
plt.colorbar(label='Kernel Value')
plt.xlabel("Sample Index")
plt.ylabel("Sample Index")
plt.tight_layout()

# Save for LaTeX inclusion
plt.savefig("figures/qsvr_kernel_matrix.png", dpi=300)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# --- Load raw data ---
FTSE = pd.read_csv('HistoricalPrices-2.csv')
BOND = pd.read_csv('Bank of England  Database-5.csv')

# --- Preprocess FTSE ---
FTSE['Date'] = pd.to_datetime(FTSE['Date'])
FTSE = FTSE.groupby([FTSE.Date.dt.year, FTSE.Date.dt.month]).tail(1)
FTSE['month_year'] = FTSE['Date'].dt.strftime('%b-%Y')
FTSE = FTSE.drop("Date", axis='columns').reset_index(drop=True).set_index('month_year')

# --- Preprocess Bond data ---
BOND['Date'] = pd.to_datetime(BOND['Date'])
BOND['BOND_Value'] = BOND['End month level of yield from British Government Stock, 20 year Nominal Par Yield              [a]             IUMLNPY']
BOND = BOND.drop("End month level of yield from British Government Stock, 20 year Nominal Par Yield              [a]             IUMLNPY", axis='columns')
BOND['month_year'] = BOND['Date'].dt.strftime('%b-%Y')
BOND = BOND.drop("Date", axis='columns').reset_index(drop=True).set_index('month_year')

# --- Join datasets ---
joined = FTSE.join(BOND)
joined['Close'] = joined[' Close']
Prepared_Data = joined.drop([' Open', ' High', ' Low', ' Close'], axis='columns')

# --- Add extra features ---
Prepared_Data['BondChange'] = Prepared_Data['BOND_Value'].diff().fillna(0)
Prepared_Data['FTSE_MA'] = Prepared_Data['Close'].rolling(window=3).mean().bfill()

# --- Plot 1: Bond yield time series ---
plt.figure(figsize=(10,4))
Prepared_Data['BOND_Value'].plot(color='tab:blue')
plt.title('UK 20-Year Treasury Bond Yield (Monthly)')
plt.xlabel('Date')
plt.ylabel('Yield (%)')
plt.tight_layout()
plt.savefig('bond_yield_timeseries.png', dpi=300)
plt.show()

# --- Plot 2: FTSE100 closing prices time series ---
plt.figure(figsize=(10,4))
Prepared_Data['Close'].plot(color='tab:green')
plt.title('FTSE100 Closing Prices')
plt.xlabel('Date')
plt.ylabel('Close Value')
plt.tight_layout()
plt.savefig('ftse_timeseries.png', dpi=300)
plt.show()

# --- Plot 3: Scatter plot ---
plt.figure(figsize=(6,6))
plt.scatter(Prepared_Data['Close'], Prepared_Data['BOND_Value'], color='tab:purple', alpha=0.7)
plt.title('Bond Yield vs FTSE100 Close')
plt.xlabel('FTSE100 Close')
plt.ylabel('20Y Bond Yield (%)')
plt.tight_layout()
plt.savefig('bond_vs_ftse_scatter.png', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Reuse your 'joined' DataFrame from preprocessing
# Convert index to datetime for cleaner tick formatting
joined.index = pd.to_datetime(joined.index, format='%b-%Y')

plt.figure(figsize=(12,6))

# Primary axis: FTSE close
ax1 = plt.gca()
ax1.plot(joined.index, joined['Close'], color='tab:green', label='FTSE100 Close')
ax1.set_xlabel('Month-Year', fontsize=12)
ax1.set_ylabel('FTSE100 Close', color='tab:green', fontsize=12)
ax1.tick_params(axis='y', labelcolor='tab:green', labelsize=10)

# Format x-axis to show one tick every 6 months
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.xticks(rotation=45, fontsize=10)

# Secondary axis: Bond yield
ax2 = ax1.twinx()
ax2.plot(joined.index, joined['BOND_Value'], color='tab:blue', label='Bond Yield (20Y)')
ax2.axhline(y=1.5, color='tab:red', linestyle='--', label='1.5% Threshold')
ax2.set_ylabel('Bond Yield (%)', color='tab:blue', fontsize=12)
ax2.tick_params(axis='y', labelcolor='tab:blue', labelsize=10)

# Title and combined legend
plt.title('FTSE100 Close and UK 20-Year Bond Yield with Coupling Weakening Threshold', fontsize=14)

# Combine legends from both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left', fontsize=10)

plt.tight_layout()
plt.savefig('figures/bond_ftse_series.png', dpi=300)
plt.show()


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# === QSVR Feature Map ===
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def feature_map_circuit(x=[0.5, 0.1]):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.RX(x[0] + 0.3, wires=0)
    qml.RY(x[1] + 0.7, wires=1)
    qml.CNOT(wires=[0,1])
    qml.RZ(x[0] * x[1], wires=1)
    qml.CNOT(wires=[0,1])
    return qml.state()

# Draw and save
drawer = qml.draw_mpl(feature_map_circuit)
fig, ax = drawer()
fig.savefig("figures/qsvr_feature_map_circuit.png", dpi=300)
plt.show()


In [ ]:
# === VQR Variational Circuit ===

n_qubits = 2
dev_vqr = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev_vqr)
def vqr_circuit(x=[0.1, 0.2], weights=np.random.uniform(0, np.pi, (3, n_qubits, 3))):
    qml.AngleEmbedding(x, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# Draw and save
drawer_vqr = qml.draw_mpl(vqr_circuit)
fig, ax = drawer_vqr()
fig.savefig("figures/vqr_variational_circuit.png", dpi=300)
plt.show()
